In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.ma as ma
from scipy.stats import linregress
import json as json
from pandas.io.json import json_normalize
import sqlite3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
import os


In [15]:
#import json file

In [16]:
filepath = os.path.join("../data/cause_of_death.json")
with open(filepath) as jsonfile:
    CODjson = json.load(jsonfile)

In [17]:
#conert to dataframe

In [18]:
CODdata = CODjson['data']
COD_df = pd.DataFrame(CODdata)
COD_df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,row-4ziw_fype-q93g,00000000-0000-0000-84F0-BE0C2CBEC77B,0,1571926784,None,1571926784,None,{ },2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,United States,169936,49.4
1,row-mf82_5qce.78c2,00000000-0000-0000-F702-31C8D4A1E5BE,0,1571926784,None,1571926784,None,{ },2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alabama,2703,53.8
2,row-7frh~9yb6-q2q4,00000000-0000-0000-6FE5-1E0821A4276B,0,1571926784,None,1571926784,None,{ },2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alaska,436,63.7
3,row-urf6.ayyb~8p5q,00000000-0000-0000-26D0-367B1AF824D1,0,1571926784,None,1571926784,None,{ },2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arizona,4184,56.2
4,row-74k2-qi6f~77p7,00000000-0000-0000-C87C-FAD80FEC2E12,0,1571926784,None,1571926784,None,{ },2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1625,51.8


In [19]:
# dataframe, all fields

In [20]:
# json file labels
# 0 - 'sid'
# 1 - 'id'
# 2 - 'position'
# 3 - 'created_at'
# *4 - 'created_meta' - null values
# 5 - 'updated_at'
# *6 - 'meta_data' - null values
# *7 - 'meta' - only contains '{}'
# 8 - 'year'
# 9 - '113 Cause Name'
# 10 - 'Cause Name'
# 11 - 'State'
# 12 - 'Deaths'
# 13 - 'Age-Adjusted Death Rate'

sid = COD_df[0]
cod_id = COD_df[1]
position = COD_df[2]
created_at = COD_df[3]
updated_at = COD_df[5]
year = COD_df[8]
exp_cause_name = COD_df[9]
cause_name = COD_df[10]
state = COD_df[11]
deaths = COD_df[12]
Age_Adj_DR = COD_df[13]

CODall_df = pd.DataFrame({"sid":sid,
                            "id":cod_id,
                            "position":position,
                            "created_at":created_at,
                            "updated_at":updated_at,
                            "year":year,
                            "113 Cause Name": exp_cause_name,
                            "Cause Name": cause_name,
                            "state": state,
                            "deaths": deaths,
                            "Age-adjusted Death Rate": Age_Adj_DR,})
CODall_df.head(10)

,sid,id,position,created_at,updated_at,year,113 Cause Name,Cause Name,state,deaths,Age-adjusted Death Rate
0,row-4ziw_fype-q93g,00000000-0000-0000-84F0-BE0C2CBEC77B,0,1571926784,1571926784,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,United States,169936,49.4
1,row-mf82_5qce.78c2,00000000-0000-0000-F702-31C8D4A1E5BE,0,1571926784,1571926784,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alabama,2703,53.8
2,row-7frh~9yb6-q2q4,00000000-0000-0000-6FE5-1E0821A4276B,0,1571926784,1571926784,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Alaska,436,63.7
3,row-urf6.ayyb~8p5q,00000000-0000-0000-26D0-367B1AF824D1,0,1571926784,1571926784,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arizona,4184,56.2
4,row-74k2-qi6f~77p7,00000000-0000-0000-C87C-FAD80FEC2E12,0,1571926784,1571926784,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1625,51.8
5,row-f3f9-wrca~275n,00000000-0000-0000-3522-4CDA1CE75FE5,0,1571926784,1571926784,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,California,13840,33.2
6,row-irx4.579t.ueij,00000000-0000-0000-49B3-681BE5DF663A,0,1571926784,1571926784,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Colorado,3037,53.6
7,row-erq8~jfjq.ri3b,00000000-0000-0000-3714-F76A946C688B,0,1571926784,1571926784,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Connecticut,2078,53.2
8,row-qwdh.rctw.wn3a,00000000-0000-0000-B558-730195983647,0,1571926784,1571926784,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Delaware,608,61.9
9,row-6878_8kbn.ipdq,00000000-0000-0000-47CF-087455AB3122,0,1571926784,1571926784,2017,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,District of Columbia,427,61.0


In [21]:
#dataframe - filtered to useful

In [22]:
CODtoDB = pd.DataFrame({"sid": sid,
                        "year":year,
                        # "113 Cause Name": exp_cause_name,
                        "Cause Name": cause_name,
                        "state": state,
                        "deaths": deaths,
                        # "Age-adjusted Death Rate": Age_Adj_DR,
                        })
CODtoDB.head(10)

,sid,year,Cause Name,state,deaths
0,row-4ziw_fype-q93g,2017,Unintentional injuries,United States,169936
1,row-mf82_5qce.78c2,2017,Unintentional injuries,Alabama,2703
2,row-7frh~9yb6-q2q4,2017,Unintentional injuries,Alaska,436
3,row-urf6.ayyb~8p5q,2017,Unintentional injuries,Arizona,4184
4,row-74k2-qi6f~77p7,2017,Unintentional injuries,Arkansas,1625
5,row-f3f9-wrca~275n,2017,Unintentional injuries,California,13840
6,row-irx4.579t.ueij,2017,Unintentional injuries,Colorado,3037
7,row-erq8~jfjq.ri3b,2017,Unintentional injuries,Connecticut,2078
8,row-qwdh.rctw.wn3a,2017,Unintentional injuries,Delaware,608
9,row-6878_8kbn.ipdq,2017,Unintentional injuries,District of Columbia,427


In [30]:
#create db, table, populate table with data from dataframe

In [31]:
import sqlite3

In [32]:
conn = sqlite3.connect('../data/causaM.sqlite')
c = conn.cursor()

In [33]:
c.execute('CREATE TABLE CoD(sid VARCHAR PRIMARY KEY, year INT, cause name VARCHAR, state VARCHAR, deaths INT))
conn.commit

SyntaxError: EOL while scanning string literal (<ipython-input-33-2f07e233db19>, line 1)

In [34]:
CODtoDB.to_sql('CoD', conn, if_exists='replace', index = False)

In [35]:
# engine = create_engine("sqlite:///causaMortis.sqlite")
# conn = engine.connect()

In [36]:
# CODtoDB.to_sql("users", conn, index=False, if_exists="replace")

In [37]:
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy import Column, Integer, String, Float

In [38]:
# Base = declarative_base()
# class User(Base):
#     __tablename__="users"
#     sid = Column("sid", primary_key=True)
#     year = Column("Year",Integer)
#     cause_name = Column("Cause Name",String(100))
#     state = Column("state", String (10))
#     deaths = Column("deaths", Integer)

In [39]:
# Base.metadata.create_all(conn)

In [40]:
# CODtoDB.to_sql(User.__tablename__, conn, index=False, if_exists="append")